In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Scraping Boliga website for data

In [145]:
import requests
from bs4 import BeautifulSoup
import re
import json

def houses_boliga(number_houses):
    """
    Returns a list of all ids for houses on boliga
    """
    house_id = list()
    url = "https://www.boliga.dk/resultat?"
    
    for i in range(int(number_houses/50)):
        new_url = url + f"?page={i}"
        response = requests.get(new_url)
        html = response.text
        soup = BeautifulSoup(html,"html.parser")
        ids = soup.find_all("a",{"class":"house-list-item"})
        link_houses = list()

        for link in ids:
            link_houses.append(re.findall("(/\d{4,}/)",link["href"])[0].replace("/",""))
        
        house_id.extend(link_houses)
    return house_id

def get_info(id_list):
    all_df = list()
    new_keys = ["registeredArea","downPayment","estateUrl","currentArchiveId","forSaleNowId",
                "foreclosureId","selfsaleEstateId","cleanStreet","estateId","latitude","longitude",
               "propertyType","priceChangePercentTotal","energyClass","price","rooms","size","lotSize",
               "floor","buildYear","city","isActive","municipality","zipCode","street","squaremeterPrice","area",
               "daysForSale","createdDate","basementSize","views"]
    
    for house_id in id_list:
        response = requests.get(f'https://api.boliga.dk/api/v2/estate/{house_id}')
        response = response.json()
        df_dict = {key: response[key] for key in new_keys}
        df = pd.DataFrame(df_dict,index=[0])
        all_df.append(df)
        
    df = pd.concat(all_df,axis=0,ignore_index=True)
    
    return df



In [146]:
df = get_info(id_houses)

In [147]:
df

,registeredArea,downPayment,estateUrl,currentArchiveId,forSaleNowId,foreclosureId,selfsaleEstateId,cleanStreet,estateId,latitude,...,isActive,municipality,zipCode,street,squaremeterPrice,area,daysForSale,createdDate,basementSize,views
0,68,225000,https://home.dk/boligkatalog/koebenhavn/1051/e...,1676546,0,0,0,Nyhavn,0,55.68044,...,True,101,1051,"Nyhavn 31B, st..",60472.0,1,53,2020-06-24T23:55:52.010Z,0,397
1,78,430000,https://www.nybolig.dk/ejerlejlighed/1051/nyha...,1688236,0,0,0,Nyhavn,0,55.68038,...,True,101,1051,Nyhavn 23 4,51777.0,1,19,2020-07-28T23:40:31.220Z,0,200
2,117,460000,http://www.skbolig.dk/sag.asp?sagsnr=221920&mg...,1672807,0,0,0,Nyhavn,0,55.67950,...,True,101,1051,"Nyhavn 12D, st.",73200.0,1,62,2020-06-16T03:15:59.387Z,0,116
3,182,625000,http://www.skbolig.dk/sag.asp?sagsnr=331320&mg...,1689738,0,0,0,Nyhavn,0,55.67973,...,True,101,1051,"Nyhavn 12, 3.",64766.0,1,13,2020-08-03T22:02:34.863Z,0,109
4,189,750000,https://www.lokalbolig.dk?sag=36-X0000157&mgl=...,1599062,0,0,0,Nyhavn,0,55.67946,...,True,101,1051,"Nyhavn 12C, st..",79338.0,1,327,2019-09-25T01:39:45.163Z,0,101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,96,290000,http://www.skbolig.dk/sag.asp?sagsnr=220520&mg...,1641880,0,0,0,Niels Hemmingsens Gade,0,55.68037,...,True,101,1153,"Niels Hemmingsens Gade 32A, 3. th.",53738.0,1,164,2020-03-05T23:03:48.983Z,0,45
496,54,190000,https://www.nybolig.dk/ejerlejlighed/1157/klos...,1679914,0,0,0,Klosterstræde,0,55.67932,...,True,101,1157,Klosterstræde 20 4,61475.0,1,45,2020-07-03T00:10:49.720Z,0,43
497,204,500000,http://www.paulun.dk/bolig-redirect?sagsnr=CI-...,1685042,0,0,0,Skindergade,0,55.67978,...,True,101,1159,"Skindergade 23, 1.",43646.0,1,27,2020-07-20T22:03:12.270Z,0,58
498,100,690000,http://www.lilienhoff.dk/redirect.htm?sag=1159...,1646535,0,0,0,Skindergade,0,55.68034,...,True,101,1159,Skindergade 15,137500.0,1,149,2020-03-20T23:03:03.000Z,50,46
